In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Set the path to your dataset directory
data_dir = 'flower_photos'

# Set the path to the directory where you want to store the split data
split_data_dir = 'split_data'

# Create directories for train, validation, and test sets
os.makedirs(os.path.join(split_data_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(split_data_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(split_data_dir, 'test'), exist_ok=True)

# Iterate over each flower category
for category in ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']:
    category_dir = os.path.join(data_dir, category)
    
    # Get the list of image filenames in the category directory
    image_files = os.listdir(category_dir)
    
    # Split the image filenames into train, validation, and test sets
    train_files, val_test_files = train_test_split(image_files, test_size=0.3, random_state=42)
    val_files, test_files = train_test_split(val_test_files, test_size=0.5, random_state=42)
    
    # Copy the image files to the respective split directories
    for split_dir, split_files in [('train', train_files), ('val', val_files), ('test', test_files)]:
        for file in split_files:
            src_path = os.path.join(category_dir, file)
            dst_path = os.path.join(split_data_dir, split_dir, category, file)
            os.makedirs(os.path.dirname(dst_path), exist_ok=True)
            shutil.copy(src_path, dst_path)

In [2]:
import tensorflow as tf

# Set the path to your split dataset directory
split_data_dir = 'split_data'

# Set the target image size
target_size = (224, 224)

# Define preprocessing function
def preprocess_image(image):
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Create a dataset from directory
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(split_data_dir, 'train'),
    image_size=target_size,
    batch_size=32,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training',
    label_mode='categorical'
)

# Apply preprocessing to the dataset
train_data = train_data.map(lambda x, y: (preprocess_image(x), y))

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(split_data_dir, 'val'),
    image_size=target_size,
    batch_size=32,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training',
    label_mode='categorical'
)

# Apply preprocessing to the dataset
val_data = val_data.map(lambda x, y: (preprocess_image(x), y))


test_data = 

Found 2567 files belonging to 5 classes.
Using 2054 files for training.
Found 551 files belonging to 5 classes.
Using 441 files for training.


In [ ]:
train_data


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)  # 5 classes for your flower dataset
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16

# Check if CUDA is available
cuda_available = tf.test.is_built_with_cuda() and tf.test.is_gpu_available(cuda_only=True)

if cuda_available:
    device = '/gpu:0'  # Use the first available GPU
    print("CUDA is available. Training on GPU.")
else:
    device = '/cpu:0'  # Use CPU
    print("CUDA is not available. Training on CPU.")

with tf.device(device):
    # Load the pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the base model layers
    base_model.trainable = False

    # Create a new model on top
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(5, activation='softmax')(x)  # 5 classes for your flower dataset
    model = tf.keras.Model(inputs, outputs)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, validation_data=val_data, epochs=10)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print(f'Test accuracy: {test_acc:.4f}')

CUDA is not available. Training on CPU.
Epoch 1/10
65/65 [==============================] - 947s 15s/step - loss: 1.5229 - accuracy: 0.3627 - val_loss: 1.6413 - val_accuracy: 0.2415
Epoch 2/10
33/65 [==============>...............] - ETA: 7:18 - loss: 1.3126 - accuracy: 0.5464

In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import VGG16

# Set the path to your dataset directory
data_dir = './flower_photos'

# Set the path to the directory where you want to store the split data
split_data_dir = './split_data'

# Create directories for train, validation, and test sets
os.makedirs(os.path.join(split_data_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(split_data_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(split_data_dir, 'test'), exist_ok=True)

# Iterate over each flower category
for category in ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']:
    category_dir = os.path.join(data_dir, category)
    
    # Get the list of image filenames in the category directory
    image_files = os.listdir(category_dir)
    
    # Split the image filenames into train, validation, and test sets
    train_files, val_test_files = train_test_split(image_files, test_size=0.3, random_state=42)
    val_files, test_files = train_test_split(val_test_files, test_size=0.5, random_state=42)
    
    # Copy the image files to the respective split directories
    for split_dir, split_files in [('train', train_files), ('val', val_files), ('test', test_files)]:
        for file in split_files:
            src_path = os.path.join(category_dir, file)
            dst_path = os.path.join(split_data_dir, split_dir, category, file)
            os.makedirs(os.path.dirname(dst_path), exist_ok=True)
            shutil.copy(src_path, dst_path)

print()
print("Done with data reading")

# Set the path to your split dataset directory
split_data_dir = 'split_data'

# Set the target image size
target_size = (224, 224)

# Define preprocessing function
def preprocess_image(image):
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Create a dataset from directory
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(split_data_dir, 'train'),
    image_size=target_size,
    batch_size=32,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training',
    label_mode='categorical'
)

# Apply preprocessing to the dataset
train_data = train_data.map(lambda x, y: (preprocess_image(x), y))

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(split_data_dir, 'val'),
    image_size=target_size,
    batch_size=32,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training',
    label_mode='categorical'
)

# Apply preprocessing to the dataset
val_data = val_data.map(lambda x, y: (preprocess_image(x), y))


test_data = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(split_data_dir, 'val'),
    image_size=target_size,
    batch_size=32,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training',
    label_mode='categorical'
)

test_data = test_data.map(lambda x, y: (preprocess_image(x), y))

print(train_data)
print()
print("Done with converting images to tensor")


# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)  # 5 classes for your flower dataset
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print()
print("Model Loaded")

# Check if CUDA is available
cuda_available = tf.test.is_built_with_cuda() and tf.test.is_gpu_available(cuda_only=True)

if cuda_available:
    device = '/gpu:0'  # Use the first available GPU
    print("CUDA is available. Training on GPU.")
else:
    device = '/cpu:0'  # Use CPU
    print("CUDA is not available. Training on CPU.")

with tf.device(device):
    # Load the pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the base model layers
    base_model.trainable = False

    # Create a new model on top
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(5, activation='softmax')(x)  # 5 classes for your flower dataset
    model = tf.keras.Model(inputs, outputs)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, validation_data=val_data, epochs=10)

print()
print("Started Training")

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print(f'Test accuracy: {test_acc:.4f}')






Done with data reading
Found 2567 files belonging to 5 classes.
Using 2054 files for training.
Found 551 files belonging to 5 classes.
Using 441 files for training.
Found 551 files belonging to 5 classes.
Using 441 files for training.
<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 5), dtype=tf.float32, name=None))>

Done with converting images to tensor

Model Loaded
CUDA is not available. Training on CPU.
Epoch 1/10
 3/65 [>.............................] - ETA: 4:36 - loss: 1.7354 - accuracy: 0.2604

KeyboardInterrupt: 